In [1]:
from lcls_live.datamaps.tabular import TabularDataMap
from lcls_live.datamaps.klystron import KlystronDataMap, klystron_pvinfo, existing_LCLS_klystrons_sector_station, subbooster_pvinfo, SUBBOOSTER_SECTORS
from pytao import Tao
import json
import os
from lcls_live import data_dir
import pandas as pd

## Build datamap for linac

In [2]:
def build_linac_dm(model):
    dat0 = [
        
        {'name': 'BC1_offset',
         'pvname':'BMLN:LI21:235:MOTR',  # mm
         'bmad_factor': 0.001,
         'bmad_name': 'O_BC1_OFFSET',
         'bmad_attribute': 'offset'
        },
        
        {'name': 'BC2_offset',
         'pvname':'BMLN:LI24:805:MOTR', # mm
         'bmad_factor': 0.001,
         'bmad_name': 'O_BC2_OFFSET',
         'bmad_attribute': 'offset'
        },
        
        {
        'name': 'L1_phase',
        'description': 'Controls the L1 phase, which is the single klystron L21_1. We will disable this for now, because the KlystronDataMap handles the phase directly.',
        'pvname': 'ACCL:LI21:1:L1S_S_PV',
        'bmad_name':'O_L1',
        'bmad_factor': 0,  # We'll disable this for now. The Klystron handles it. 
        'bmad_attribute':'phase_deg'
        
    }
    ]
    
    dat_hxr = [
            {
        'name': 'L2_phase',
        'pvname': 'ACCL:LI22:1:PDES',
        'bmad_name':'O_L2',
        'bmad_factor': 1,
        'bmad_attribute':'phase_deg'
        
    },
        {
        'name': 'L3_phase',
        'pvname': 'ACCL:LI25:1:PDES',
        'bmad_name':'O_L3',
        'bmad_attribute':'phase_deg',
        'bmad_offset': 0
        
    }, 
    ]
    
    # SXR has different PVs
    dat_sxr = [
            {
        'name': 'L2_phase',
        'pvname': 'ACCL:LI22:1:PDES:SETDATA_1',
        'bmad_name':'O_L2',
        'bmad_factor': 1,
        'bmad_attribute':'phase_deg'
        
    },
        {
        'name': 'L3_phase',
        'pvname': 'ACCL:LI25:1:PDES:SETDATA_1',
        'bmad_name':'O_L3',
        'bmad_attribute':'phase_deg',
        'bmad_offset': 0
        
    }, 
    ]

    #Note that there are sone NaNs here. That's okay.
    if model == 'cu_hxr':
        df = pd.DataFrame(dat0+dat_hxr)
    elif model == 'cu_sxr':
        df = pd.DataFrame(dat0+dat_sxr)
    else:
        raise ValueError(f'Unknown model: {model}')
        
    dm = TabularDataMap(df, pvname='pvname', element='bmad_name', attribute='bmad_attribute', factor='bmad_factor', offset='bmad_offset')
    
    return dm

## Build datamaps for klystrons

In [3]:
def build_klystron_dms(tao, model):
    
    if model == 'cu_hxr':
        beamcode = 1
    elif model == 'cu_sxr':
        beamcode = 2
    else:
        raise ValueError(f'Unknown model: {model}')
    
    
    klystron_names = tao.lat_list('overlay::K*', 'ele.name', flags='-no_slaves')

    klystron_datamaps = []
    for sector, station in existing_LCLS_klystrons_sector_station:
        info = klystron_pvinfo(sector, station, beamcode=beamcode)
        k = KlystronDataMap(**info)

        if k.name in klystron_names:
            klystron_datamaps.append(k)
    
    return klystron_datamaps

## Build datamaps for quadrupoles

In [4]:
def quad_pvinfo(tao, ele):
    """
    Returns dict of PV information for use in a DataMap
    """
    head = tao.ele_head(ele)
    attrs = tao.ele_gen_attribs(ele)
    device = head['alias']
    
    d = {}
    d['bmad_name'] = ele
    d['pvname_rbv'] = device+':BACT'
    d['pvname'] = device+':BDES'    
    d['bmad_factor'] = -1/attrs['L']/10
    d['bmad_attribute'] = 'b1_gradient'
    return d

def build_quad_dm(tao):
    quad_names = tao.lat_list('quad::*', 'ele.name', flags='-no_slaves')
    dfq = pd.DataFrame([quad_pvinfo(tao, ele) for ele in quad_names])
    dm = TabularDataMap(dfq, pvname='pvname', element='bmad_name', attribute = 'bmad_attribute', factor='bmad_factor')
    return dm

## Build datamaps for collimators

## Build datamap for energy measurements

In [5]:
def build_energy_dm(model):
    # The syntax is flexible enough to use for getting measurements for Tao
    ENERGY_MEAS0 = [
        {
        'name': 'L1_energy',
        'pvname': 'BEND:LI21:231:EDES', # or EDES
        'tao_datum': 'BC1.energy[1]',        
        'factor': 1e9
        },
        {
        'name': 'L2_energy',
        'pvname': 'BEND:LI24:790:EDES', # or EDES
        'tao_datum': 'BC2.energy[1]',
        'factor': 1e9
        }
    ]
    
    
    ENERGY_MEAS_HXR = [ {
        'name': 'L3_HXR_energy',
        'pvname': 'BEND:DMPH:400:EDES', # or EDES
        'tao_datum': 'L3.energy[2]',
        'factor': 1e9
        } ]
    
    ENERGY_MEAS_SXR = [ {
        'name': 'L3_SXR_energy',
        'pvname': 'BEND:DMPS:400:EDES', # or EDES
        'tao_datum': 'L3.energy[2]',
        'factor': 1e9
        } ]    
    
    if model == 'cu_hxr':
        df = pd.DataFrame(ENERGY_MEAS0 + ENERGY_MEAS_HXR)
    elif model == 'cu_sxr':
         df = pd.DataFrame(ENERGY_MEAS0 + ENERGY_MEAS_SXR)
    else:
        raise ValueError(f'Unknown model: {model}')    
    
    
    dm = TabularDataMap(df, pvname='pvname', element='tao_datum', factor='factor',
                       tao_format = 'set data {element}|meas  = {value}',
                       bmad_format = '! No equivalent Bmad format for: set data {element}|meas  = {value}'
                       )
    return dm

## Build subbooster datamap

In [6]:
def build_subbooster_dm():
    subboosters = []
    for sector in SUBBOOSTER_SECTORS:

        dat = subbooster_pvinfo(sector) 
        dat['bmad_name'] = f'SBST_{sector}'
        dat['bmad_attribute'] = 'phase_deg'
        subboosters.append(dat)
    df = pd.DataFrame(subboosters)    

    # Make the DataMap object, identifying the columns to be used
    dm = TabularDataMap(df, pvname='phase_pvname', element='bmad_name', attribute='bmad_attribute')
    return dm

## Beginning Twiss datamap function

In [7]:
def beginning_meas_twiss_datamap(name, pvprefix):
    dat =  [
    {
    'name': f'{name}_beta_x_meas',
    'pvname': f'{pvprefix}:BETA_X', 
    'bmad_name': 'beginning',   
    'bmad_attribute': 'beta_a'
    },
    {
    'name': f'{name}_beta_y_meas',
    'pvname': f'{pvprefix}:BETA_Y', 
    'bmad_name': 'beginning',   
    'bmad_attribute': 'beta_b'
    },
    {
    'name': f'{name}_alpha_x_meas',
    'pvname': f'{pvprefix}:ALPHA_X', 
    'bmad_name': 'beginning',   
    'bmad_attribute': 'alpha_a'
    },
    {
    'name': f'{name}_alpha_y_meas',
    'pvname': f'{pvprefix}:ALPHA_Y', 
    'bmad_name': 'beginning',   
    'bmad_attribute': 'alpha_b'
    },    
    ]
    
    df= pd.DataFrame(dat)

    return TabularDataMap(df, pvname='pvname', element='bmad_name', attribute = 'bmad_attribute')

## Utility function for creating map for given model 

In [8]:
def build_json_rep(model):
    #set up tao
    # Basic model with options
    MODEL = model
    OPTIONS = f'-slice OTR2:END -noplot'
    INIT = f'-init $LCLS_LATTICE/bmad/models/{MODEL}/tao.init {OPTIONS}'
    tao = Tao(INIT)
    
    
    rep = []
    
    #subbooster_dm
    dm = build_subbooster_dm()
    rep.append(
        {
        "name": "subboosters",
        "class": "tabular",
        "data": dm.to_json()
        }
    )
    
    #linac
    dm = build_linac_dm(model)
    rep.append(
        {
        "name": "linac",
        "class": "tabular",
        "data": dm.to_json()
        }
    )
    
    #klystron
    dms = build_klystron_dms(tao, model)
    for dm in dms:
        rep.append(
            {
            "name": dm.name,
            "class": "klystron",
            "data": dm.to_json()
            }
        )
    
    
    #quad
    dm = build_quad_dm(tao)
    rep.append(
        {
        "name": "quad",
        "class": "tabular",
        "data": dm.to_json()
        }
    )
    
    #otr2
    dm = beginning_meas_twiss_datamap('WS02', 'WIRE:IN20:561' )
    rep.append(
        {
        "name": "beginning_WS02",
        "class": "tabular",
        "data": dm.to_json()
        }
    )
    
    
    #energy_meas
    dm = build_energy_dm(model)
    rep.append(
        {
        "name": "tao_energy_measurements",
        "class": "tabular",
        "data": dm.to_json()
        }
    )
    
    return rep

In [9]:
cu_hxr = build_json_rep("cu_hxr")
cu_sxr = build_json_rep("cu_sxr")

## Dump to file

In [10]:
with open(os.path.join(data_dir, 'datamaps_master.json'), "w") as f:
    json.dump({"cu_hxr": cu_hxr, "cu_sxr": cu_sxr}, f)